In [ ]:
import pandas as pd
import numpy as np
import difflib
from math import cos, radians
from haversine import haversine, Unit
import folium




In [97]:
#INPUT CELL:
input_recipe = input("Please enter a recipe (e.g. Pasta Putanesca): ")
Lat_in = 35.2220
Lon_in = -101.8313


Please enter a recipe (e.g. Pasta Putanesca): chicken fontina


In [98]:
# dear robert
# this is Ermias from class
# modify these file directories for the ones I am going to upload
# something like
# data_cleaned/USDA_file.csv

farm_in = "USDA_file.csv"
recipe_in = "Recipe_file.csv"
farms = pd.read_csv(farm_in)
recipes = pd.read_csv(recipe_in)


In [99]:
recipes['title'] = recipes['title'].astype(str)
recipes['matched_ingredient'] = recipes['matched_ingredient'].astype(str)
farms['LAT'] = farms['LAT'].astype(float)
farms['LON'] = farms['LON'].astype(float)

In [100]:
rec_list = recipes.title.unique().tolist()
hey = rec_list[:1000]
match_list = difflib.get_close_matches(input_recipe, rec_list, n=3, cutoff=0.1)


In [101]:
match_df = recipes[recipes['title'].isin(match_list)]
ingredient_list = match_df.matched_ingredient.unique().tolist()


In [102]:
farms['INGREDIENT']=farms['INGREDIENT'].apply(lambda x: x.lower())

In [103]:
farm_search = farms[farms['INGREDIENT'].isin(ingredient_list)]


In [115]:


farm_search.loc[:,'es'] = farm_search.apply(lambda x: haversine((Lat_in,Lon_in),(x.LAT, x.LON),unit='mi'), axis=1)



In [105]:
def my_agg(x):
    names = {
        'score': x['es'].min()}

    return pd.Series(names, index=['score'])

idx = farm_search.groupby(['INGREDIENT'])['es'].transform(min) == farm_search['es']
farm_vals = farm_search[idx]
alpha = farm_vals.groupby(['INGREDIENT','LAT','LON']).apply(my_agg).reset_index()
beta = alpha.set_index(['INGREDIENT'])



In [106]:
def score_ex(x,y):
    try:
        return beta.loc[x][y]
    except:
        return None

match_df.loc[:,'score'] = match_df.loc[:,'matched_ingredient'].apply(lambda x: score_ex(x,'score'))
match_df.loc[:,'Lat'] = match_df.loc[:,'matched_ingredient'].apply(lambda x: score_ex(x,'LAT'))
match_df.loc[:,'Lon'] = match_df.loc[:,'matched_ingredient'].apply(lambda x: score_ex(x,'LON'))


In [107]:


scores = match_df.groupby('title').score.agg(['mean']).sort_values('mean', ascending=True).reset_index()
winner = scores.title.iloc[0]
print('The closest matching recipe is: ' + winner)
closest_df = match_df[match_df['title']==winner][['matched_ingredient','Lat','Lon']]
closest_2 = closest_df


The closest matching recipe is: Chicken Athena 


In [108]:
closest_df = closest_df.dropna().reset_index()[['matched_ingredient','Lat','Lon']].reset_index()
closest_df.head()

,index,matched_ingredient,Lat,Lon
0,0,olives,32.217942,-97.769211
1,1,chicken,35.398675,-101.893804
2,2,garlic,34.962529,-101.895547
3,3,fresh cut herbs,34.572984,-103.346055
4,4,tomatoes,34.962529,-101.895547


In [109]:
frame_in = pd.DataFrame({'index': [50], 'matched_ingredient': ['Home'], 'Lat': [Lat_in], 'Lon': [Lon_in]})


In [112]:

         
m=folium.Map(location=[Lat_in,Lon_in])
for item in range(len(closest_df)):
    df_new = pd.concat([closest_df[closest_df['index'] == item], frame_in])
    
    points = list(df_new[['Lat','Lon']].to_records(index=False))
    #print(df_new["position_lat_degrees"].head())
    
    try:
        folium.PolyLine(locations=points,weight=5).add_to(m)
        folium.Marker([df_new["Lat"].iloc[0],df_new["Lon"].iloc[0]], 
                      popup=df_new['matched_ingredient'].iloc[0]).add_to(m)
        
    except: 
        print('Farm to Face Map has been Canned')
    #m.add_children(route)
m.fit_bounds([[closest_df.Lat.min(), closest_df.Lon.min()], [closest_df.Lat.max(), closest_df.Lon.max()]])
display(m)